<center><b><font size=6>Text data pre-processing</font></b></center>

In this exercice, we shall load a database of email messages and pre-format them so that we can design automated classification methods or use off-the-shelf classifiers.

"What is there to pre-process?" you might ask. Well, actually, text data comes in a very noisy form that we, humans, have become accustomed to and filter out effortlessly to grasp the core meaning of the text. It has a lot of formatting (fonts, colors, typography...), punctuation, abbreviations, common words, grammatical rules, etc. that we might wish to discard before even starting the data analysis.

Here are some pre-processing steps that can be performed on text:
1. loading the data, removing attachements, merging title and body;
2. tokenizing - splitting the text into atomic "words";
3. removal of stop-words - very common words;
4. removal of non-words - punctuation, numbers, gibberish;
3. lemmatization - merge together "find", "finds", "finder".

The final goal is to be able to represent a document as a mathematical object, e.g. a vector, that our machine learning black boxes can process.

# Load the data

Let's first load the emails.

In [1]:
import os
data_switch=1
if(data_switch==0):
    train_dir = '../data/ling-spam/train-mails/'
    email_path = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
else:
    train_dir = '../data/lingspam_public/bare/'
    email_path = []
    email_label = []
    for d in os.listdir(train_dir):
        folder = os.path.join(train_dir,d)
        email_path += [os.path.join(folder,f) for f in os.listdir(folder)]
        email_label += [f[0:3]=='spm' for f in os.listdir(folder)]
print("number of emails",len(email_path))
email_nb = 0 # try 8 for a spam example
print("email file:", email_path[email_nb])
print("email is a spam:", email_label[email_nb])
print(open(email_path[email_nb]).read())

number of emails 2893
email file: ../data/lingspam_public/bare/part9/9-56msg1.txt
email is a spam: False
Subject: aiml ' 98 : first call for papers

first call for papers aiml ' 98 advances in modal logic ' 98 october 16-18 , 1998 uppsala university , uppsala , sweden advances in modal logic is an initiative aimed at presenting an up-to - date picture of the state of the art in modal logic and its many applications . the initiative consists of a workshops series together with volumes based on those workshops . advances in modal logic ' 98 is the second workshop organized as part of this initiative . aiml ' 98 will be held from october 16-18 , 1998 in uppsala , sweden . the workshop is intended for users of modal logic in cognition , computing , and language , as well as for logicians working in modal logic . topics . aiml ' 98 will be organized around a number of thematic areas : - modal logics of agency and normative systems - algebraic and model-theoretic aspects of modal logic - mod

# Filtering out the noise

One nice thing about scikit-learn is that is has lots of preprocessing utilities. Like [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for instance, that converts a collection of text documents to a matrix of token counts.

- To remove stop-words, we set: `stop_words='english'`
- To convert all words to lowercase: `lowercase=True`
- The default tokenizer in scikit-learn removes punctuation and only keeps words of more than 2 letters.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer(input='filename', stop_words='english', lowercase=True)
word_count = countvect.fit_transform(email_path)

In [3]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names_out()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 60618
Document - words matrix: (2893, 60618)
First words: ['00' '000' '0000' '00001' '00003000140' '00003003958' '00007' '0001'
 '00010' '00014' '0003' '00036' '000bp' '000s' '000yen' '001' '0010'
 '0010010034' '0011' '00133' '0014' '00170' '0019' '00198' '002' '002656'
 '0027' '003' '0030' '0031' '00333' '0037' '0039' '003n7' '004' '0041'
 '0044' '0049' '005' '0057' '006' '0067' '007' '00710' '0073' '0074'
 '00799' '008' '009' '00919680' '0094' '00a' '00am' '00arrival' '00b'
 '00coffee' '00congress' '00d' '00dinner' '00f' '00h' '00hfstahlke' '00i'
 '00j' '00l' '00m' '00p' '00pm' '00r' '00t' '00tea' '00the' '00uzheb' '01'
 '0100' '01003' '01006' '0104' '0106' '01075' '0108' '011' '0111' '0117'
 '0118' '01202' '01222' '01223' '01225' '01232' '01235' '01273' '013'
 '0131' '01334' '0135' '01364' '0139' '013953' '013a']


# Even better filtering

That's already quite ok, but this pre-processing does not perform lemmatization, the list of stop-words could be better and we could wish to remove non-english words (misspelled, with numbers, etc.).

A slightly better preprocessing uses the [Natural Language Toolkit](https://www.nltk.org/https://www.nltk.org/). The one below:
- tokenizes;
- removes punctuation;
- removes stop-words;
- removes non-English and misspelled words (optional);
- removes 1-character words;
- removes non-alphabetical words (numbers and codes essentially).

In [4]:
# If this is the first time you run this notebook, run the lines below
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/emmanuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/emmanuel/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/emmanuel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk import wordpunct_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
class LemmaTokenizer(object):
    def __init__(self, remove_non_words=True):
        self.wnl = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('english'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.wnl.lemmatize(t) for t in word_list]

countvect = CountVectorizer(input='filename',tokenizer=LemmaTokenizer(remove_non_words=True))
word_count = countvect.fit_transform(email_path)
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

/home/emmanuel/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names_out()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 14279
Document - words matrix: (2893, 14279)
First words: ['aa' 'aal' 'aba' 'aback' 'abacus' 'abandon' 'abandoned' 'abandonment'
 'abbas' 'abbreviation' 'abdomen' 'abduction' 'abed' 'aberrant'
 'aberration' 'abide' 'abiding' 'abigail' 'ability' 'ablative' 'ablaut'
 'able' 'abler' 'aboard' 'abolition' 'abord' 'aboriginal' 'aborigine'
 'abound' 'abox' 'abreast' 'abridged' 'abroad' 'abrogate' 'abrook'
 'abruptly' 'abscissa' 'absence' 'absent' 'absolute' 'absolutely'
 'absoluteness' 'absolutist' 'absolutive' 'absolutization' 'absorbed'
 'absorption' 'abstract' 'abstraction' 'abstractly' 'abstractness'
 'absurd' 'absurdity' 'abu' 'abundance' 'abundant' 'abuse' 'abusive'
 'abyss' 'academe' 'academic' 'academically' 'academician' 'academy'
 'accelerate' 'accelerated' 'accelerative' 'accent' 'accentuate'
 'accentuation' 'accept' 'acceptability' 'acceptable' 'acceptance'
 'acceptation' 'accepted' 'acception' 'access' 'accessibility'
 'accessible' 'acce

# Term frequency times inverse document frequency

After this first preprocessing, each document is summarized by a vector of size "number of words in the extracted dictionnary". For example, the first email in the list has become:

In [7]:
mail_number = 0
text = open(email_path[mail_number]).read()
print("Original email:")
print(text)
#print(LemmaTokenizer()(text))
#print(len(set(LemmaTokenizer()(text))))
#print(len([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(len([word_count2[mail_number, i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(set([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]])-set(LemmaTokenizer()(text)))
emailBagOfWords = {feat2word[i]: word_count[mail_number, i] for i in word_count[mail_number, :].nonzero()[1]}
print("Bag of words representation (", len(emailBagOfWords), " words in dict):", sep='')
print(emailBagOfWords)
print("\nVector reprensentation (", word_count[mail_number, :].nonzero()[1].shape[0], " non-zero elements):", sep='')
print(word_count[mail_number, :])

Original email:
Subject: aiml ' 98 : first call for papers

first call for papers aiml ' 98 advances in modal logic ' 98 october 16-18 , 1998 uppsala university , uppsala , sweden advances in modal logic is an initiative aimed at presenting an up-to - date picture of the state of the art in modal logic and its many applications . the initiative consists of a workshops series together with volumes based on those workshops . advances in modal logic ' 98 is the second workshop organized as part of this initiative . aiml ' 98 will be held from october 16-18 , 1998 in uppsala , sweden . the workshop is intended for users of modal logic in cognition , computing , and language , as well as for logicians working in modal logic . topics . aiml ' 98 will be organized around a number of thematic areas : - modal logics of agency and normative systems - algebraic and model-theoretic aspects of modal logic - modal approaches to grammar and natural language semantics - computational aspects of modal 

Counting words is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called `tf` for Term Frequencies.

Another refinement on top of `tf` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called `tf–idf` for “Term Frequency times Inverse Document Frequency” and again, scikit-learn does the job for us with the [TfidfTransformer](scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) function.

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit_transform(word_count)
tfidf.shape

(2893, 14279)

Now every email in the corpus has a vector representation that filters out unrelevant tokens and retains the significant information.

In [9]:
print("email 0:")
print(tfidf[0,:])

email 0:
  (0, 14175)	0.025349465817376148
  (0, 14174)	0.22506638374675217
  (0, 14171)	0.02861304122922822
  (0, 14071)	0.030600125583418934
  (0, 14002)	0.06375456355210782
  (0, 14001)	0.02294172630882425
  (0, 13971)	0.024614522450428524
  (0, 13851)	0.09057496564981124
  (0, 13731)	0.06939013872305265
  (0, 13668)	0.0694904308067751
  (0, 13635)	0.051525781519962066
  (0, 13427)	0.0466135924895318
  (0, 12893)	0.031300605458185284
  (0, 12856)	0.030253334925539985
  (0, 12845)	0.028157936923182542
  (0, 12725)	0.04757110280385832
  (0, 12718)	0.0981968155803024
  (0, 12171)	0.03063230621438087
  (0, 12170)	0.08552986404007432
  (0, 12153)	0.018503844147659414
  (0, 11991)	0.06810300050430651
  (0, 11983)	0.06641617566846952
  (0, 11947)	0.0255148471795215
  (0, 11892)	0.05216409819060909
  (0, 11784)	0.05233548640354513
  :	:
  (0, 2286)	0.05321260635579905
  (0, 2261)	0.04222277157587959
  (0, 1960)	0.033836026972513276
  (0, 1733)	0.042065911335269796
  (0, 1418)	0.042567603616

# Utility function

Let's put all this loading process into a separate file so that we can reuse it in other experiments.

In [10]:
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()

In [11]:
spam_data.print_email(8)

email file: ../data/lingspam_public/bare/part9/8-923msg1.txt
email is a spam: False
Subject: non - profit linguistic web site

the web site linguistic enterprises is now available at http : / / web . gc . cuny . edu / dept / lingu / enter / this is a non-profit site that aims to help academically trained linguists find private sector employment . it offers down-to - earth advice , how-to information , and an opportunity to discuss prospects and problems with others who have found work or are seeking it . one section of the site is designed to match those wanting linguistic jobs in the private sector with companies looking to hire language specialists . the site is maintained by the ph . d . program in linguistics at the graduate school , city university of new york , in conjunction with the linguistic society of america . please tell others about this forum . if you know of potential employers please encourage them to post opportunities for linguists . if you ' re looking for a job , p